<a href="https://colab.research.google.com/github/parijatgaur/Colabprojects/blob/main/DeepNN_Pytorch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import matplotlib.colors as ListedColormap
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as f


torch.manual_seed(1)


In [2]:
# Define the function to plot the diagram

def plot_decision_regions_3class(model, data_set):
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#00AAFF'])
    cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#00AAFF'])
    X = data_set.x.numpy()
    y = data_set.y.numpy()
    h = .02
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    XX = torch.Tensor(np.c_[xx.ravel(), yy.ravel()])
    _, yhat = torch.max(model(XX), 1)
    yhat = yhat.numpy().reshape(xx.shape)
    plt.pcolormesh(xx, yy, yhat, cmap=cmap_light)
    plt.plot(X[y[:] == 0, 0], X[y[:] == 0, 1], 'ro', label = 'y=0')
    plt.plot(X[y[:] == 1, 0], X[y[:] == 1, 1], 'go', label = 'y=1')
    plt.plot(X[y[:] == 2, 0], X[y[:] == 2, 1], 'o', label = 'y=2')
    plt.title("decision region")
    plt.legend()

In [3]:
# data creation

# Create Data Class

class Data(Dataset):

    #  modified from: http://cs231n.github.io/neural-networks-case-study/
    # Constructor
    def __init__(self, K=3, N=500):
        D = 2
        X = np.zeros((N * K, D)) # data matrix (each row = single example)
        y = np.zeros(N * K, dtype='uint8') # class labels
        for j in range(K):
          ix = range(N * j, N * (j + 1))
          r = np.linspace(0.0, 1, N) # radius
          t = np.linspace(j * 4, (j + 1) * 4, N) + np.random.randn(N) * 0.2 # theta
          X[ix] = np.c_[r * np.sin(t), r*np.cos(t)]
          y[ix] = j
        self.y = torch.from_numpy(y).type(torch.LongTensor)
        self.x = torch.from_numpy(X).type(torch.FloatTensor)
        self.len = y.shape[0]

    # Getter
    def __getitem__(self, index):
        return self.x[index], self.y[index]

    # Get Length
    def __len__(self):
        return self.len

    # Plot the diagram
    def plot_stuff(self):
        plt.plot(self.x[self.y[:] == 0, 0].numpy(), self.x[self.y[:] == 0, 1].numpy(), 'o', label="y = 0")
        plt.plot(self.x[self.y[:] == 1, 0].numpy(), self.x[self.y[:] == 1, 1].numpy(), 'ro', label="y = 1")
        plt.plot(self.x[self.y[:] == 2, 0].numpy(), self.x[self.y[:] == 2, 1].numpy(), 'go', label="y = 2")
        plt.legend()

In [6]:
class NN(nn.Module):
  def __init__(self, layers):
    super(NN,self).__init__()
    self.hidden = nn.ModuleList()
    for i in range(len(layers)-2):
      self.hidden.append(nn.Linear(layers[i], layers[i+1]))

# self.hidden = nn.ModuleList([ nn.Linear( layers[i],layers[i+1] ) for i in range(len(layers - 1))])
  def forward(self, activation):
    for i, layer in enumerate(self.hidden):
      if i < len(self.hidden)-1:
        activation = f.relu(layer(activation))
      else:
        activation = layer(activation)
        return activation




In [14]:
def train(dataset, model, criterion, train_loader, optimizer, epochs=100):
  i = 0
  Loss = []
  Acc = []
  us = { "Total Loss": Loss, " Accuracy ": Acc  }
  for e in range(epochs):
    for x, y in train_loader:
      optimizer.zero_grad()
      yhat = model(x)
      loss = criterion(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      Loss.append(loss.item())
      fig, ax1 = plt.subplots()
  color = 'tab:red'
  ax1.plot(Loss, color = color)
  ax1.set_xlabel('Iteration', color = color)
  ax1.set_ylabel('total loss', color = color)
  ax1.tick_params(axis = 'y', color = color)

  ax2 = ax1.twinx()
  color = 'tab:blue'
  ax2.set_ylabel('accuracy', color = color)  # we already handled the x-label with ax1
  ax2.plot(Acc, color = color)
  ax2.tick_params(axis = 'y', color = color)
  fig.tight_layout()  # otherwise the right y-label is slightly clipped

  plt.show()
  return Loss






In [11]:
def accuracy( model, data_set):
  _,yhat = torch.max(model(data_set.x), 1)
  return (yhat == data_set.y).float().mean()



In [ ]:
# Create a Dataset object

data_set = Data()
data_set.plot_stuff()
data_set.y = data_set.y.view(-1)

layers = [2,50,3]
learning_rate = 0.1
model = NN(layers )
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
train_loader = DataLoader( dataset = data_set, batch_size = 100)

LOSS = train(data_set, model, criterion, train_loader, optimizer, epochs=100)





<ipython-input-14-862a1d21e4c7>:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax1 = plt.subplots()
